# Boosting


In [1]:
install.packages("data.table")


Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/kj/dkjqkk2n3wq2zfbttgdpjrj80000gn/T//RtmpAyOwuu/downloaded_packages


In [2]:
list_of_packages <- c ("synthpop", "insight", "party", "dplyr", "rpart", "rpart.plot", "randomForest", "pROC", "caret", "pracma", "here", "Hmisc", "arf", "randomForest", "caret", "xgboost", "data.table")

lapply(list_of_packages, FUN= function(X){
  do.call("require", list(X))
})

Lade n"otiges Paket: synthpop

Find out more at https://www.synthpop.org.uk/

Lade n"otiges Paket: insight

Lade n"otiges Paket: party

Lade n"otiges Paket: grid

Lade n"otiges Paket: mvtnorm

Lade n"otiges Paket: modeltools

Lade n"otiges Paket: stats4

Lade n"otiges Paket: strucchange

Lade n"otiges Paket: zoo


Attache Paket: 'zoo'


Die folgenden Objekte sind maskiert von 'package:base':

    as.Date, as.Date.numeric


Lade n"otiges Paket: sandwich

Lade n"otiges Paket: dplyr


Attache Paket: 'dplyr'


Das folgende Objekt ist maskiert 'package:party':

    where


Die folgenden Objekte sind maskiert von 'package:stats':

    filter, lag


Die folgenden Objekte sind maskiert von 'package:base':

    intersect, setdiff, setequal, union


Lade n"otiges Paket: rpart

Lade n"otiges Paket: rpart.plot

Lade n"otiges Paket: randomForest

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attache Paket: 'randomForest'


Das folgende Objekt ist maskiert 'package:dpl

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] FALSE

[[14]]
[1] TRUE

[[15]]
[1] TRUE

[[16]]
[1] FALSE

[[17]]
[1] TRUE

## Data

In [3]:
load(file = (paste0(here(), "/cpspop.RData")))
adult <- read.csv(file = (paste0(here(),"/adult_preprocessed.csv")))
# delete NAs
adult[adult == "?"] <- NA
adult <- na.omit(adult)

adult$workclass <- as.factor(adult$workclass)
adult$education <- as.factor(adult$education)
adult$marital_status <- as.factor(adult$marital_status)
adult$relationship <- as.factor(adult$relationship)
adult$race <- as.factor(adult$race)
adult$sex <- as.factor(adult$sex)
adult$native_country <- as.factor(adult$native_country)
adult$income <- as.factor(adult$income)

In [9]:
head(adult)

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
,<int>,<fct>,<int>,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>,<fct>
1,82,2,132870,11,6,3,1,4,0,0,4356,18,38,<=50K
2,54,2,140359,5,0,6,4,4,0,0,3900,40,38,<=50K
3,41,2,264663,15,5,9,3,4,0,0,3900,40,38,<=50K
4,34,2,216864,11,0,7,4,4,0,0,3770,45,38,<=50K
5,38,2,150601,0,5,0,4,4,1,0,3770,40,38,<=50K
6,74,5,88638,10,4,9,2,4,0,0,3683,20,38,>50K


### Helper functions

In [6]:
## eval cont targets
evaluation_metrics_cont <- function(predictions, test_set){
  MAE <- mean(abs(predictions - test_set$income))
  MSE <- mean((predictions - test_set$income)^2)
  RMSE <- sqrt(MSE)

  SS_res <- sum((test_set$income - predictions)^2)
  SS_tot <- sum((test_set$income - mean(test_set$income))^2)
  R_squared <- 1 - (SS_res / SS_tot)

  MAPE <- mean(abs((test_set$income - predictions) / test_set$income)) * 100

  # Create the dataframe
  metrics_df <- data.frame(
        MAE = MAE, 
        MSE = MSE, 
        RMSE = RMSE,
        R_squared = R_squared, 
        MAPE = MAPE)

  return(metrics_df)
}

In [7]:
## eval factored targets
evaluation_metrics_factor <- function(predictions, test_set){
    # confusion matrix for the prediction on original data
    cm <- confusionMatrix(predictions, test_set$income,
                mode = "everything")

    # saving evaluation metrics
    accuracy <- cm$overall['Accuracy']
    f1 <- cm$byClass['F1']
    sens <- cm$byClass['Sensitivity']
    spec <- cm$byClass['Specificity']

    # Create the dataframe
    metrics_df <- data.frame(
        Accuracy = accuracy, 
        F1 = f1, 
        Sensitivity = sens, 
        Specificity = spec
    )
    
    return(metrics_df)
}

In [4]:
convert_to_numeric_matrix <- function(df) {
  # Convert factors to numeric
  df[] <- lapply(df, function(col) {
    if (is.factor(col)) {
      return(as.numeric(col))
    } else {
      return(col)
    }
  })
  
  # Convert the entire data frame to a matrix
  matrix_data <- as.matrix(df)
  return(matrix_data)
}

In [5]:
cps_matrix <- convert_to_numeric_matrix(cpspop)
adult_matrix <- convert_to_numeric_matrix(adult)

In [19]:
target_col <- 'income'
target_col_index <- which(colnames(cps_matrix) == target_col)

# Split the data into features (X) and target (y)
X <- cps_matrix[, -target_col_index]
y <- cps_matrix[, target_col_index]

# Convert to DMatrix
dtrain <- xgb.DMatrix(data = X, label = y)

# Set parameters for XGBoost
params <- list(
  objective = "reg:squarederror",  # Use "reg:squarederror" for regression tasks or change accordingly
  max_depth = 3,
  eta = 0.1
)

# Train the model
bst_model <- xgb.train(params = params, data = dtrain, nrounds = 100)

syn_xgb <- function(model, X, n_samples) {
  synthetic_data <- list()
  for (i in 1:n_samples) {
    # Sample a random row from the feature matrix
    random_sample <- X[sample(1:nrow(X), 1), , drop = FALSE]
    
    # Predict the target variable for the sampled row
    prediction <- predict(model, xgb.DMatrix(data = random_sample))
    
    # Combine the random sample with the predicted target
    synthetic_sample <- as.data.frame(random_sample)
    synthetic_sample$income <- prediction  # Rename the target column to 'income'
    
    # Append to the list
    synthetic_data[[i]] <- synthetic_sample
  }
  
  # Combine all synthetic samples into a data frame
  synthetic_df <- do.call(rbind, synthetic_data)
  return(synthetic_df)
}

       tax csp age educ marital race sex    ss   income
48035 1800   0  37    9       1    1   1     0 68356.65
40955  125   0  39   13       1    1   2     0 76002.25
25483    0   0  48   11       5    1   2     0 34738.66
49967  324   0  59    9       1    1   1 13326 43434.59
41719    0   0  25   10       1    1   2     0 43361.99
25519  900   0  61    9       1    1   1     0 58998.41


In [ ]:
# Generate synthetic data
syn_cps <- syn_xgb(bst_model, X, nrow(cpspop))

print(head(synthetic_data))